In [1]:
import numpy as np
import scipy.sparse as sp

为方便叙述，我们先假定矩阵$A = (a_1, a_2, \cdots, a_n)$是列满秩的。由GS过程我们知道分解过程如下：
1. $R_{11} \leftarrow \|a_1\|_2$
2. $q_1 \leftarrow \frac{a_1}{R_{11}}$
3. for $i = 2, \cdots, n$:
    1. $R_{ji} \leftarrow a_i^\top q_j$ for $j = 1, \cdots, i-1$
    2. $q_i \leftarrow a_i - \sum_{j=1}^{i-1} R_{ji}q_j$
    3. $R_{ii} \leftarrow \|q_i\|_2$
    4. $q_i \leftarrow \frac{q_i}{R_{i}}$

在数学上，以上算法是没有问题的。但在实际操作层面，由于计算机能表示的精度有限，在计算投影的时候以上算法会存在较大的数值问题，即$q_i, q_j$的相互垂直性由于浮点运算而产生较大偏离。在矩阵$A$接近于不满秩的时候，这个问题尤为严重。一个巧妙的降低最终浮点运算误差的方法是更换计算次序，从而使得计算投影时产生的误差不会被反复累加，具体过程如下：
1. $R_{11} \leftarrow \|a_1\|_2$
2. $q_1 \leftarrow \frac{a_1}{R_{11}}$
3. for $i = 2, \cdots, n$
    1. $q_i \leftarrow a_i$
    2. for $j = 1, \cdots, i - 1$:
        1. $R_{ji} \leftarrow q_i^\top q_j$
        2. $q_i \leftarrow q_i - R_{ji}q_j$
    3. $R_{ii} \leftarrow \|q_i\|_2$
    4. $q_i \leftarrow \frac{q_i}{R_{i}}$

我们将第一个算法称为Classical Gram-Schmidt(cgs)，第二个算法称为Modified Gram-Schmidt(mgs)。

下面的代码实现的是对一般的矩阵$A$（不需要列满秩）实现reduced form QR分解。

In [2]:
def qr(A, method, reduced=True):
    eps = 1e-8
    m, n = A.shape
    AT = A.T
    if not reduced:
        raise NotImplementedError
    if method == "cgs":
        q = None
        r_row, r_col, r_data = [], [], []

        for i in range(n):
            if q is None:
                rii = (AT[i] * AT[i]).sum() ** .5
                if rii > eps:
                    r_row.append(0)
                    r_col.append(i)
                    r_data.append(rii)
                    q = AT[i] / rii
            else:
                rji = q.dot(AT[i])
                qi = AT[i] - q.T.dot(rji)
                rii = (qi * qi).sum() ** .5
                length = len(rji) if len(rji.shape) > 0 else 1
                rs = list(rji) if length > 1 else [rji]
                if rii > eps:
                    r_row.extend(list(range(length + 1)))
                    r_col.extend([i] * (length + 1))
                    r_data.extend(rs + [rii])
                    q = np.vstack((q, qi / rii))
                else:
                    r_row.extend(list(range(length)))
                    r_col.extend([i] * length)
                    r_data.extend(rs)
        R = sp.coo_matrix(
            (r_data, (r_row, r_col)),
            shape=(len(q), n)
        ).toarray()
        Q = q.T
        return Q, R
    elif method == "mgs":
        qlist = []
        r_row, r_col, r_data = [], [], []

        for i in range(n):
            qi = AT[i]
            Rjilist = []
            for q in qlist:
                Rji = qi.dot(q)
                Rjilist.append(Rji)
                qi = qi - Rji * q
            r_row.extend(list(range(len(Rjilist))))
            r_col.extend([i] * len(Rjilist))
            r_data.extend(Rjilist)
            rii = (qi * qi).sum() ** .5
            if rii > eps:
                r_row.append(len(Rjilist))
                r_col.append(i)
                r_data.append(rii)
                qlist.append(qi / rii)
        R = sp.coo_matrix(
            (r_data, (r_row, r_col)),
            shape=(len(qlist), n)
        ).toarray()
        Q = np.asarray(qlist).T
        return Q, R
    else:
        raise NotImplementedError

下面一个例子可以直观感受到GS过程中的数值问题。

我们首先产生一个高度奇异的矩阵$A$（每一列都是相同的向量加上一个小的随机扰动），然后对其进行QR分解。

In [3]:
A = np.random.randn(30, 20) * 1e-6 + np.arange(20)
Q1, R1 = qr(A, method="cgs")
Q2, R2 = qr(A, method="mgs")

我们令
- decomp err = $\max_{i,j} |A_{ij} - (QR)_{ij}|$，用来表示分解之后和原矩阵的差异
- orthogonal err = $\max_{i,j} |I_{ij} - (Q^\top Q)_{ij}|$，用来表示分解之后矩阵$Q$对单位正交性的偏离

结果如下，mgs算法使得正交性数值问题得到缓解。

In [4]:
print("cgs decomp err = {:.3e}".format(
    np.abs(A - Q1.dot(R1)).max()))
print("mgs decomp err = {:.3e}".format(
    np.abs(A - Q2.dot(R2)).max()))
print("cgs orthogonal err = {:.3e}".format(
    np.abs(Q1.T.dot(Q1) - np.eye(Q1.shape[1])).max()))
print("mgs orthogonal err = {:.3e}".format(
    np.abs(Q2.T.dot(Q2) - np.eye(Q2.shape[1])).max()))

cgs decomp err = 1.066e-14
mgs decomp err = 7.105e-15
cgs orthogonal err = 3.874e-02
mgs orthogonal err = 3.176e-09


事实上，由于GS过程需要计算投影而引入较大数值误差，在成熟算法库中不使用该算法进行QR分解。替代的算法有Householder、Givens等，本质上是用旋转来代替投影，来缓解数值问题。